<a href="https://colab.research.google.com/github/menezesglesia/Tarefas-P-s/blob/main/NLTK_NaiveBayes_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode

     |████████████████████████████████| 235 kB 4.9 MB/s 


In [ ]:
# Load EDA Pkgs
import pandas as pd
import numpy as np
import random
from unidecode import unidecode
import string
import re


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stop = stopwords.words('portuguese')
# stop.append('nao')
stop2 = list()
for word in stop:
  stop2.append(unidecode(word))
  
#stemmer = nltk.stem.RSLPStemmer()

In [ ]:
def strip_all_entities(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


In [ ]:
def strip_links(text):
  return re.sub(r"https:\/\/t.co\/.*", '', text)

def add_space_punctuation(text):
  s = re.sub('([.,!?#@$%&:\'\"()])', r' \1 ', text)
  s = re.sub('\s{2,}', ' ', s)
  s = re.sub(r";", '', s)
  s = re.sub(r" a ", ' ', s)
  s = re.sub(r" n ", ' ', s)
  return s

In [ ]:
df_nbr = pd.read_csv('amostras-revisado.csv', delimiter=';')
df_nbr = df_nbr.loc[df_nbr['sentiment_revised'].ne(0)]

In [ ]:
# remoção de palavras de parada
df_nbr['text_treated'] = df_nbr['text'].str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_nbr['text_treated'] = df_nbr['text_treated'].str.lower().apply(lambda x: strip_links(x))
df_nbr['text_treated'] = df_nbr['text_treated'].str.lower().apply(lambda x: add_space_punctuation(x))
df_nbr['text_treated'] = df_nbr['text_treated'].str.lower().apply(lambda x: unidecode(x))
#df_nbr['text_treated'] = df_nbr['text_treated'].str.lower().apply(lambda x: strip_all_entities(x))
df_nbr['text_treated'] = df_nbr['text_treated'].str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in (stop2)]))

#df_nbr['text_treated'] = df_nbr['text_treated'].str.replace('[{}]'.format(string.punctuation), ' ')
df_nbr['text_treated'] = df_nbr['text_treated'].str.replace('[{}]'.format(string.digits), ' ')
print(df_nbr['text'][6])
print("-------")
print(df_nbr['text_treated'][6])

Hahahahah... Acabei de maratonar #VerdadeSecretas2 todos os capítulos disponíveis na @globoplay , pq sou dessas...… https://t.co/sNvRCVmrbO
-------
hahahahah . . . acabei maratonar # verdadesecretas  todos capitulos disponiveis @ globoplay , pq dessas . . . ...


In [ ]:
df_nbr['words'] = df_nbr['text_treated'].apply(lambda x: word_tokenize(x))
  
all_words = nltk.FreqDist([word.lower() for words in df_nbr['words'].to_list() for word in words])
print(all_words)
# len(all_words)
# word_features = list(all_words)[:200] 
#word_features = list(all_words.most_common()[0])[:500]
word_features = [x[0] for x in all_words.most_common(1000)]

<FreqDist with 1048 samples and 2211 outcomes>


In [ ]:

[x[0] for x in all_words.most_common(20)]

['@',
 '.',
 ',',
 'netflix',
 '...',
 '!',
 '?',
 'globoplay',
 'disney',
 'pra',
 '#',
 'vou',
 ':',
 'vai',
 'hbo',
 '``',
 'q',
 'pq',
 'max',
 'serie']

In [ ]:
def find_features(document):    
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features    

In [ ]:
def label_it(x):
  if x > 0:
    return "pos"
  else:
    return 'neg'

df_nbr['sentiment_label_revised'] = df_nbr['sentiment_revised'].apply(lambda x: label_it(x))


In [ ]:
#df_nbr['sentiment_revised'].to_list()
words = df_nbr['words'].to_list()
sentiments = df_nbr['sentiment_label_revised'].to_list()
documents = [(words[i], sentiments[i]) for i, sentiment in enumerate(sentiments)]

In [ ]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]
# set that we'll train our classifier with
training_set = featuresets[:150]

# set that we'll test against.
testing_set = featuresets[150:]
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 79.16666666666666


In [ ]:
classifier.show_most_informative_features(20)

Most Informative Features
                   serie = True              pos : neg    =      6.6 : 1.0
                   todos = True              pos : neg    =      4.6 : 1.0
                  filmes = True              pos : neg    =      4.6 : 1.0
                     ter = True              neg : pos    =      3.7 : 1.0
                     boa = True              pos : neg    =      3.6 : 1.0
                       & = True              pos : neg    =      3.6 : 1.0
                  melhor = True              pos : neg    =      3.6 : 1.0
                      ta = True              pos : neg    =      3.4 : 1.0
                     vai = True              neg : pos    =      3.0 : 1.0
                       ? = True              neg : pos    =      2.7 : 1.0
                  disney = True              neg : pos    =      2.7 : 1.0
                       ! = True              pos : neg    =      2.6 : 1.0
                     obg = True              pos : neg    =      2.6 : 1.0

In [ ]:
# dist = classifier.prob_classify(training_set)
# for label in dist.samples():
#     print("%s: %f" % (label, dist.prob(label)))
#classifier.show_most_informative_features(10)
for label in classifier.labels():
  print(f'\n\n{label}:', end='')
  print("{0:.2f}%".format(100*classifier._label_probdist.prob(label)))
  
  for (fname, fval) in classifier.most_informative_features(50):
    print(f"   {fname}({fval}): ", end="")
    print("{0:.2f}%".format(100*classifier._feature_probdist[label, fname].prob(fval)))


In [ ]:
#classifier.classify(find_features('nao acredito que vou ter que assinar a globoplay'.split()))
# print((find_features('Vou ter que assinar a globoplay. Nao to acreditando.'.lower().split())))
res = classifier.prob_classify (find_features(unidecode('Todos os filmes e series são boa').lower().split()))
list(res.samples())
print(f"{res.prob('neg')} negative")
print(f"{res.prob('pos')} positive")

0.0719326019126703 negative
0.9280673980873311 positive


In [ ]:
pos = len([x for x in training_set if x[1] == 'pos'])
neg = len([x for x in training_set if x[1] == 'neg'])

print(f"{pos/len(training_set)}")
print(f"{neg/len(training_set)}")

print(training_set)

0.3933333333333333
0.6066666666666667
[({'@': False, '.': False, ',': False, 'netflix': False, '...': False, '!': False, '?': False, 'globoplay': True, 'disney': False, 'pra': True, '#': False, 'vou': True, ':': False, 'vai': False, 'hbo': False, '``': False, 'q': True, 'pq': False, 'max': False, 'serie': False, 'assistir': True, 'ta': False, 'to': False, 'ter': True, 'ver': False, 'fazer': False, 'filme': False, '(': False, "'": False, 'bem': False, 'acho': False, 'ruim': False, ')': False, 'nunca': False, 'nada': False, 'secretas': True, 'todos': False, 'gente': False, 'boa': False, 'vida': False, 'alguem': False, 'queria': False, '-': False, 'filmes': False, 'melhor': False, 'hoje': False, 'ser': False, 'tempo': False, 'tudo': False, 'verdades': True, 'kkkkkk': False, 'assisti': False, 'ai': False, 'realmente': False, 'agora': False, '&': False, 'noite': False, 'desse': False, 'tao': False, 'ainda': False, 'sei': False, 'obg': False, 'you': False, 'melhores': False, 'mc': False, 'da